In [1]:
from IPython.display   import Image , display
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.graph  import StateGraph   , START , END
from langgraph.graph import MessagesState
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode

import os 
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
model = ChatGroq(
    model="llama-3.1-8b-instant",   
    api_key=os.getenv("GROQ_API_KEY"),
    temperature=0
)

In [3]:
@tool
def multiply(x:int , y:int)-> int:
    """Returns the multiplication of given two integers"""
    return x*y

@tool
def add(x:int , y:int)->int:
    """Returns the addition of given two integers """
    return x+y 

@tool
def divide(x:int , y:int)-> int:
    """Returns the division of given two integers"""
    return x/y

In [4]:
tools=[multiply , add , divide]
llm_with_tools= model.bind_tools(tools)

In [5]:
sys_msg=SystemMessage(content= "You are a helpful assisstant tasked with performing arithmetic actions on a  given set of inputs")

In [6]:
def assistant(state:MessagesState):
    return {"messages":[llm_with_tools.invoke([sys_msg] + state["messages"] ) ] }


In [7]:
builder= StateGraph(MessagesState)
builder.add_node("tools" , ToolNode(tools))
builder.add_node("assistant" , assistant)

builder.add_edge(START , "assistant")
builder.add_conditional_edges("assistant" , tools_condition)
builder.add_edge("tools" , "assistant")
#react_graph=builder.compile()

#display(Image(react_graph.get_graph().draw_mermaid_png()))

In [8]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()
react_graph_memory= builder.compile(checkpointer= memory)

In [9]:
config = {"configurable": {"thread_id":"1"}}

messages= [HumanMessage(content = "add 3 and 4")]

messages = react_graph_memory.invoke({"messages":messages} , config)
for m in messages['messages']:
    m.pretty_print()



================================ Human Message =================================

add 3 and 4
================================== Ai Message ==================================
Tool Calls:
  add (r48gpfrt5)
 Call ID: r48gpfrt5
  Args:
    x: 3
    y: 4
================================= Tool Message =================================
Name: add

7
================================== Ai Message ==================================

The result of the addition of 3 and 4 is 7.


In [10]:
messages= [HumanMessage(content = "Multiply that by 2")]

messages = react_graph_memory.invoke({"messages":messages} , config)
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

add 3 and 4
================================== Ai Message ==================================
Tool Calls:
  add (r48gpfrt5)
 Call ID: r48gpfrt5
  Args:
    x: 3
    y: 4
================================= Tool Message =================================
Name: add

7
================================== Ai Message ==================================

The result of the addition of 3 and 4 is 7.
================================ Human Message =================================

Multiply that by 2
================================== Ai Message ==================================
Tool Calls:
  multiply (fs6ze75qq)
 Call ID: fs6ze75qq
  Args:
    x: 7
    y: 2
================================= Tool Message =================================
Name: multiply

14
================================== Ai Message ==================================

The result of the multiplication of 7 and 2 is 14.
